<a href="https://colab.research.google.com/github/thdgmltjd123/NenepBigData/blob/master/preprocessing/DatasetPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브와 코랩 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import copy
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import emoji
import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec

In [ ]:
label = os.listdir("testDatas")

In [ ]:
label

['actionadventure',
 'animation',
 'biography',
 'crimeaction',
 'horror',
 'mysterythriller',
 'romanticcomedy',
 'scifi',
 'war']

In [ ]:
labelled_data = []
la = labelled_data.append
for gidx, genre in enumerate(label):
    la(labeling_data(os.listdir("testDatas\\"+genre),"testDatas\\"+genre+"\\", gidx))

In [ ]:
genre_df = []
for i in range(9):
    genre_df.append(labelled_data[i][0].append(labelled_data[i][1:]))

dataset = genre_df[0].append(genre_df[1:],sort = False)

In [ ]:
dataset = dataset.sample(frac = 1).reset_index(drop = True)
dataset = dataset.dropna().reset_index(drop = True)
len(dataset)

146879

In [ ]:
def labeling_data(folder_list, path, label):
    df_list = []
    for movie in folder_list:
        data = pd.read_csv(path+movie, names = ["idx", "review"])
        data["label"] = label
        data = data.iloc[1:,1:]
        df_list.append(data)
    return df_list

In [ ]:
path = os.listdir("reviewDatas")
def load_data(folder_list, path):
    movie_review = []
    ma = movie_review.append
    for movie in folder_list:
        data = pd.read_csv(path+movie, names = ["idx", "review"])
        data = data.iloc[1:,1:]
        review_list = []
        ra = review_list.append
        for i in range(len(data)):
            ra(data.iloc[i,0])
        ma(review_list)
    return movie_review

In [ ]:
def preprocessing(df):
    review_list = list(df["review"])
    multi_emoji = re.compile(r'[:]{2,}[\w_-]+')
    shortword = re.compile(r'\W*\b\w{1,2}\b')
    punc = re.compile(r'[\W_]+')
    first_upper = re.compile(r'\s*[A-Z][a-z0-9]+')
    digit = re.compile(r'\d+')

    for i in tqdm(range(len(review_list))):
        try:
            review_list[i] = multi_emoji.sub('',emoji.demojize(review_list[i]))
            review_list[i] = shortword.sub('',review_list[i])
            review_list[i] = punc.sub(' ',review_list[i])
            review_list[i] = first_upper.sub('',review_list[i])
            review_list[i] = digit.sub('',review_list[i])
            review_list[i] = shortword.sub('',review_list[i])
            review_list[i] = review_list[i].lower()
            review_list[i] = review_list[i].strip()
        except:
            print(i)

        df["review"] = review_list
    return df

In [ ]:
df = preprocessing(dataset)

100%|██████████| 146879/146879 [1:51:10<00:00, 22.02it/s] 


In [ ]:
df.to_csv("preprocessing.csv")

In [ ]:
df = pd.read_csv("preprocessing.csv", index_col = 0)

In [ ]:
#불용어 목록 만들기
def make_stopwords() :
    stop_words = set(stopwords.words("english"))
    google_sw = "a about above after again against all am an and any are aren't as at be because been before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most mustn't my myself no nor not of off on once only or other ought our ours ourselves out over own same shan't she she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't you you'd you'll you're you've your yours yourself yourselves"
    google_sw = set(google_sw.split(' '))
    no_upper_sw = set("arent cant couldnt didnt doesnt dont hadnt hasnt hed hes heres hows id ill im ive isnt lets mustnt shant shed shes shouldnt thats theres theyd theyll theyre theyve wasnt wed weve werent whats whens wheres whos whys wont wouldnt youd youll youre youve".split(' '))

    stop_words = stop_words | google_sw | no_upper_sw
    return stop_words

In [ ]:
def elim_stopwords(df):
    stop_words = make_stopwords()
    review_list = list(df["review"])
    for idx, review in enumerate(review_list):
        li = re.split("\s+",review)
        li = [item for item in li if item not in stop_words]
        review_list[idx] = " ".join(li)
        
    df["review"] = review_list
    print("eliminating stopwords is done --> pos tagging")
    
    return df

In [ ]:
df=df.dropna().reset_index(drop=False)
df = elim_stopwords(df)

eliminating stopwords is done --> pos tagging


In [ ]:
len(df)

146718

In [ ]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [ ]:
def tagging_lemmatize(df, min_words = 5):
    n=WordNetLemmatizer()
    review_list = list(df["review"])

    for idx, review in enumerate(tqdm(review_list)):
        li = review.split(" ")
        if len(li) > min_words:
            li = pos_tag(li)
            for k, token in enumerate(li):
                try:
                    li[k] = n.lemmatize(token[0],get_wordnet_pos(token[1]))
                except KeyError:
                    li[k] = n.lemmatize(token[0])
        review_list[idx] = " ".join(li)
    df["review"] = review_list
    elim_idx = df[df["review"]==''].index
    df = df.drop(elim_idx)
    
    return df.reset_index(drop = True)

In [ ]:
df = tagging_lemmatize(df)
df = df.iloc[:,1:]

100%|██████████| 146718/146718 [27:00<00:00, 90.51it/s] 


In [ ]:
df.to_csv("lemmatizing.csv")

In [ ]:
review_train, review_test, label_train, label_test = train_test_split(dataset.iloc[:,0],dataset.iloc[:,1], test_size = 0.3, stratify = dataset.iloc[:,1], random_state = 777)

5    7674
3    7308
7    6813
0    5215
2    4801
8    4541
1    3396
4    2869
6    1447
Name: label, dtype: int64

## 전처리 방안

+ 이모티콘을 단어로 변환 후 중복된 이모티콘을 1개로 만들기   
+ 단어의 길이가 1에서 2인 것 제거    
+ 리뷰의 구두점들을 제거      
+ 첫번째 단어가 대문자거나 대문자로 되어있는 모든 단어들 제거   
+ 숫자 제거      
+ 제거되지 않은 대문자 모임 소문자화 후 양끝 공백제거

In [ ]:
all_token = []
aa = all_token.append
for idx,review_list in enumerate(movie_review):
    token_list = preprocessing(review_list)
    aa(token_list)
    if (idx+1) % 3 == 0:
        print("{}%".format((idx+1) / len(movie_review) * 100), end = " ")

10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% 100.0% 

all_token 리스트의 길이 30개(영화 수)    
all_token\[0\]는 첫번째 영화의 token_list  

## 불용어 처리

추가사항   
구글의 stopwords english와 합집합.   
리뷰에 의도적으로 어퍼스토로피를 빼고 작성한 경우 존재.   
stopwords가 작동을 하지 않음.    
따라서 적당히 단어가 되지 않는 선에서 stopwords 추가   

In [ ]:
all_token = elim_stopwords(all_token)

# 제거해야 하는 리뷰들    

+ 첫번째 글자를 항상 대문자로 작성하였음...     
    
데드풀 938,"Deadpool, Was The Perfect Comic Book Movie, Ryan Reynolds Was Born To Play The Character, And The Movie Is A Very Good Different Kind Of Movie, It's Fun To Watch! Has Great Jokes It's Fun! Funny! Has Great Wallbreakers! Good Amount Of Action! You Don't Want The Movie To End It's Been A While Sense I Seen A Superhero That's This Good And Perfect! Ryan Reynolds Is Inspiring In This Movie Makes You Wanna Become An Actor So You Can Play A Character Like This! This Movie Deserves To Be On Imdbs Top Movies, It Can't Get Better Than That, Hoping To See More Deadpool In The Future, One Of The Best Superhero/Antiheros Ever! Movie Had A Simple But Fun Story!"    
    
기생충 759,"I'm An Indian.I Watched It After A Lot Of Recommendation,But It Never Fullfilled My Expectations.The Movie Shows The Division In Between Peoples Such As High Class And Low Class,It Presents In The Way Of A Comedy Drama.I'm Not Satisfied On The Movie Maybe Because Of I'd Watch Lot Of Indian Movies Which Has These Kind Of Story."     
    
Ludo 67 "A Nice Movie Which Has Everything In It. If Creators Cut Duration By 30 Minutes Then It's Became More Enjoyable."    
     
Mulan 1337,"I Dislike When People Compare The Mulan Live Action Movie To The Original Animated Movie There Two Totally Different   Movies.I Thought The Cinematography Was Absolutely Astounding,The Characters Where Amazing And The Special Effects Where Great."   
    
SooraraiPottru 287 "Good Direction Fabulous Acting Good Cinematography"    
    
스파이더맨 146,"Animation 10/10     
Story 10/10     
Direction 10/10     
Script 10/10     
Imagination 100/10Loved it...."     

TheTrialoftheChicago7 25, "Performances: 10/10 Pacing: 7/10 Plot: 9/10 Writing: 10/10 Cinematography: 7/10 Soundtrack: 7/10        Enjoyability: 8/10"         
    
나크나로크 907,Niiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiicccccccccccccccccccccccccccceeeeeeeeeeeeeeeeeeeeeeeeeeee     

# Pos-tagging    

품사 태깅

In [ ]:
all_token = pos_tagging(all_token)

progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%


# Stemming and Lemmatization

감성단어는 보통 형용사가 많기 때문에    
Stemming은 과한 일반화가 될 수 있으므로 시행하지 않음   
pos-tagging(품사 태깅) 후 Lemmatization만 시행

In [ ]:
all_token[8][759]

[('never', 'RB'),
 ('seen', 'VBN'),
 ('movie', 'NN'),
 ('every', 'DT'),
 ('little', 'JJ'),
 ('detail', 'NN'),
 ('dialogue', 'NN'),
 ('object', 'NN'),
 ('foreshadows', 'VBZ'),
 ('things', 'NNS'),
 ('come', 'VBP'),
 ('fabulous', 'JJ'),
 ('way', 'NN'),
 ('made', 'VBD'),
 ('film', 'NN'),
 ('entertaining', 'VBG'),
 ('lots', 'NNS'),
 ('suspense', 'JJ'),
 ('end', 'NN')]

In [ ]:
all_token = lemmatization(all_token)

progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%


all_token\[0\~29\] --> 영화 0번부터 29번까지    
all_token\[0\]\[0\~k\] --> 0번 영화의 0~k번째 리뷰의 토큰 리스트

In [ ]:
path = os.listdir(r"testDatas\actionadventure")

In [ ]:
path

['Apocalypto.csv',
 'AvengersEndgame.csv',
 'EnolaHolmes.csv',
 'Gladiator.csv',
 'IntheHeartoftheSea.csv',
 'KingArthur.csv',
 'KingdomofHeaven.csv',
 'KingKong.csv',
 'RobinHoodPrinceofThieves.csv',
 'Shichininnosamurai.csv',
 'SnowWhiteandtheHuntsman.csv',
 'Superman.csv',
 'ThePoseidonAdventure.csv',
 'TheRevenant.csv',
 'Wilow.csv']